In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import ee
import geemap
from datetime import datetime
from pathlib import Path
# import os
from osgeo import gdal

import rioxarray
import numpy as np
import glob

In [2]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

# Designate area of interest (AOI)

## Set up name and directory structure

This is where you can change the ID of the watershed

In [ ]:
# If you're doing watersheds
HYBAS_ID = 8100362730																						
aoi_name = str(HYBAS_ID)

# If you're not
#aoi_name = 'test_aoi'

## Set up area

### Example: lat/long with area buffer

In [ ]:
aoi = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_10").filter(ee.Filter.eq('HYBAS_ID', HYBAS_ID))

longitude = aoi.geometry().centroid().coordinates().get(0).getInfo()
latitude = aoi.geometry().centroid().coordinates().get(1).getInfo()

You can use geemap's `geemap.shp_to_ee()` function to turn a local shapefile into an AOI as well. 

# Define EE functions and get filtered ImageCollection

## Functions

define bands that will be used in functions

In [ ]:
redBand = 'SR_B4'
greenBand = 'SR_B3'
blueBand = 'SR_B2'

NIR_band = 'SR_B5'
SWIR_band = 'SR_B6'

In [ ]:
# def maskL8clouds(image):
#         """Masks clouds based on pixel quality attributes

#     Parameters:
#     image (Image): A single Image in an ImageCollection or standalone Image

#     Returns:
#     Image with masked features and original metadata

#     """
#         qa = image.select('QA_PIXEL')

#         # Bits 3 and 4 are cloud and cloud shadow, respectively.
#         cloudShadowBitMask = 1 << 4
#         cloudBitMask = 1 << 3

#         # Both flags should be set to zero, indicating clear conditions.
#         mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
#             .And(qa.bitwiseAnd(cloudBitMask).eq(0))

#         return image.updateMask(mask) \
#             .divide(10000) \
#             .copyProperties(image, ['system:time_start'])

In [ ]:
# cloud and shadow mask
def maskL8clouds(image):
    mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)

    return image.updateMask(mask) \
            .divide(10000) \
            .copyProperties(image, ['system:time_start'])

In [ ]:
def clp(image):
    '''Clips a single Image to a region of interest'''
    return image.clip(aoi)

def mosaicByDate(imcol):
    # Get a list of unique dates in the image collection
    unique_dates = ee.List(imcol.aggregate_array('system:time_start')).map(
        lambda date: ee.Date(date).format('YYYY-MM-dd')
    ).distinct()
    
    def mosaic_date(date):
        date = ee.Date(date)
        filtered = imcol.filterDate(date, date.advance(1, 'day'))
        return ee.Image(filtered.mosaic()).set('system:time_start', date.millis())
    
    mosaic_imlist = unique_dates.map(mosaic_date)
    return ee.ImageCollection(mosaic_imlist)

def addNDVI(image):
  '''Adds NDVI band to each image (in an ImageCollection)'''
  ndvi = image.normalizedDifference([NIR_band, redBand]).rename('NDVI')
  return image.addBands(ndvi)

def addNDSI(image):
    '''Adds L8's NDSI band to each image (in an ImageCollection)'''
    ndsi = image.normalizedDifference([greenBand, SWIR_band]).rename('NDSI')
    return image.addBands(ndsi)

def addElevation(image):
  '''Adds ArcticDEM elevation to each image (in an ImageCollection)'''
  elevation = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").select('elevation').clip(aoi).rename('ArcticDEM')
  return image.addBands(elevation)

def get_mean_snow_cover(image):
        """Adds a value for scene-averaged MODIS-dervied snow cover to an image (in an ImageCollection)

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with snow cover mean as a band

        """    
        # Get MODIS snow cover product for day and location
        ndsi_image = ee.ImageCollection('MODIS/061/MOD10A1').filterDate(
            image.date(), image.date().advance(1, 'day')).first().select('NDSI_Snow_Cover').clip(aoi)
        
        image = image.addBands(ndsi_image)

        # Get mean value across the scene 
        mean_value = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=100,  # Resolution of Landsat data in meters
            maxPixels = 1e9
        )

        # Get the mean value for the band
        mean_band_value = mean_value.get('NDSI_Snow_Cover')

        # Set the mean value as an image property
        return image.set("mean_" + 'NDSI_Snow_Cover', mean_band_value)

def get_white_pixel_percent(image):
        """Add data on pixel percentage that is white in grayscale as a band
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "white_percentage" band added

    """    
        grayscale = image.expression(
            '(.3 * R) + (.59 * G) + (.11 * B)', {
            'R': image.select(redBand),
            'G': image.select(greenBand),
            'B': image.select(blueBand)
        })

        white_mask = grayscale.gt(2000)
        
    
        white_pixel_stats = white_mask.reduceRegion(
        reducer=ee.Reducer.sum().combine(
                reducer2=ee.Reducer.count(),
                sharedInputs=True
            ),
            geometry=aoi,  # Assuming 'aoi' is defined somewhere in your script
            scale=30,
            maxPixels=1e9
        )

        # Extract sum and count values
        sum_white_pixels = ee.Number(white_pixel_stats.get('sum'))
        count_white_pixels = ee.Number(white_pixel_stats.get('count'))

        # Calculate percentage
        white_percentage = sum_white_pixels.divide(count_white_pixels).multiply(100)

        return image.set("white_percentage", white_percentage)

def calcTotalPixels(image):
    """Add data on total pixels as a band

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "total_pixels" band added

    """    
    total_pixels = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.count(),
        scale=30,  
        maxPixels = 1e9,
        geometry=aoi
    )
    return image.set("total_pixels", total_pixels) 


In [ ]:
# original
def calculateNoDataPercentage(image):
        """Add data on masked pixel percentage as a property
        note that total_pixels needs to be calculated first

    Parameters:
    image (Image): A single Image in an ImageCollection or standalone Image

    Returns:
    Image with "nodata_percentage property set"

    """
        pixel_area = ee.Image.pixelArea()
        
        nodata_mask = image.select('QA_PIXEL').eq(0)
        nodata_area = pixel_area.updateMask(nodata_mask)
        
        nodata_pixel_count = nodata_area.reduceRegion(
              reducer=ee.Reducer.sum(),
              geometry=aoi,
              scale=30,
              maxPixels=1e9
            ).get('QA_PIXEL')
        
        nodata_pixels = ee.Number(nodata_pixel_count)
        total_pixels = ee.Number(image.get(total_pixels))
        
        # Calculate percentage
        nodata_percentage = nodata_pixels.divide(total_pixels).multiply(100)
        
        # Add NoData percentage as a property
        return image.set('nodata_percentage', nodata_percentage)



In [ ]:
# def addCloudScore(image):
#     cloud_score = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
#     return image.addBands(cloud_score)

# def filterClouds(image):
#     cloud_score = image.select('cloud').reduceRegion(
#         reducer=ee.Reducer.mean(),
#         geometry=image.geometry(),
#         scale=30
#     )
#     return ee.Number(cloud_score.get('cloud')).lte(20) # change mean cloud score threshold

In [ ]:
def filterCloudyScenes(imcol, cloud_threshold):
     return imcol.filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold))

In [ ]:
def applyScaleFactors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

## Set filtering parameters 

If you notice that your ImageCollections are empty, try changing these!

In [ ]:
cloud_threshold = 10

# Seward 
snow_cover_threshold = 5
threshold_nodata_percent = 5
threshold_white_percent = 5

# Limit images added to the Map
image_limit=5


## Build ImageCollection

This step involves loading in Sentinel 2 data and applies your first filter for dates and cloudy percentage as well as filtering to images that intersect with the bounds of your area of interest and masking for clouds. 

For detecting water tracks we want to look at the growing season (months 5 to 9) but adjust based on science question. 

In [ ]:
cloudyScenes = (
                ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') 
                .filter(ee.Filter.calendarRange(2019,2023,'year'))
                .filter(ee.Filter.calendarRange(6,8,'month'))
                .filterBounds(aoi)
                .map(clp)
                .map(applyScaleFactors)
)

pre_filter = filterCloudyScenes(cloudyScenes, cloud_threshold)

dataset = pre_filter.map(maskL8clouds)



In [ ]:
dataset = mosaicByDate(dataset).map(addNDVI).map(addNDSI).map(addElevation)

#collection_with_data = (dataset.map(calcTotalPixels).map(calculateNoDataPercentage).map(get_white_pixel_percent))


In [ ]:
# Basically a "too many aggregated requests" error can be thrown if you do TOO much at once (like three reduceRegions and filters)
# So my strategy here is to subsume the snow filter into the white filter

#filtered_collection = collection_with_data.filter(ee.Filter.lte("nodata_percentage", threshold_nodata_percent))
#filtered_collection = filtered_collection.filter(ee.Filter.lte("white_percentage", threshold_white_percent))\

# NOT TESTING FILTERS^ FOR SPECTRAL CHANGE PURPOSES
filtered_collection = dataset

filtered_collection_size = len(filtered_collection.aggregate_array("system:index").getInfo())

print(f'Number of images in collection: {filtered_collection_size}')

In [ ]:
Map = geemap.Map(center=[latitude, longitude], zoom=11)

In [ ]:
for image_id in filtered_collection.aggregate_array("system:index").getInfo()[0:image_limit]:
    image = filtered_collection.filterMetadata("system:index", "equals", image_id).first()
    
    image_RGB = image.select(redBand, greenBand, blueBand) 
    # image_RGB = image.select('B4') 
    RGB_vis_params = {'min': 0.0, 'max': 0.15, 'gamma': 1.4}
    Map.addLayer(image_RGB, RGB_vis_params, ee.Image(image).date().format('yyyy-MM-dd').getInfo(), True)

print(f'Remember I am limiting the image list to {image_limit} of {filtered_collection_size} images')


Now if you click on the wrench in the right top corner, followed by the "Layers" button, you can toggle through imagery that passed the filter. 

If you notice squares missing from the imagery, it might be a rendering/tiling issue - if you zoom in and out the map will re-tile and the problem is usually solved. The data are there, I promise. 


# Map

In [ ]:
Map